In [1]:
!pip install --user google-cloud-bigquery==1.25.0

     |████████████████████████████████| 169 kB 7.9 MB/s            
     |████████████████████████████████| 93 kB 1.2 MB/s             
     |████████████████████████████████| 152 kB 59.8 MB/s            
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
google-cloud-storage 1.43.0 requires google-resumable-media<3.0dev,>=1.3.0; python_version >= "3.6", but you have google-resumable-media 0.5.1 which is incompatible.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.33.0 which is incompatible.


In [2]:
import os
from google.cloud import bigquery

In [3]:
%%bash 
gcloud config list project --format "value(core.project)"

harsh-bqml


In [4]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: harsh-bqml


In [5]:
%%bash

# Create a BigQuery dataset for babyweight if it doesn't exist
datasetexists=$(bq ls -d | grep -w nyc_bike_share)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset already exists, let's not recreate it."

else
    echo "Creating BigQuery dataset titled: nyc_bike_share"
    
    bq --location=US mk --dataset \
        --description "nyc bike share data" \
        $PROJECT:nyc_bike_share
    echo "Here are your current datasets:"
    bq ls
fi

Creating BigQuery dataset titled: nyc_bike_share
Dataset 'harsh-bqml:nyc_bike_share' successfully created.
Here are your current datasets:
    datasetId     
 ---------------- 
  nyc_bike_share  


## Check data quality

In [8]:
%%bigquery

SELECT
    COUNT(tripduration) AS num_of_trips, 
    MIN(tripduration) AS lowest_trip_duration, 
    MAX(tripduration) AS highest_trip_duration,
    AVG(tripduration) AS average__trip_duration,  
    STDDEV(tripduration) AS stddev_trip_duration
FROM 
    `bigquery-public-data.new_york_citibike.citibike_trips`

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.79s/rows]


,num_of_trips,lowest_trip_duration,highest_trip_duration,average__trip_duration,stddev_trip_duration
0,53108721,60,19510049,962.490986,13546.093563


In [6]:
%%bigquery

SELECT 
    COUNT(*)
FROM 
    `bigquery-public-data.new_york_citibike.citibike_trips`
WHERE
    tripduration is NULL OR tripduration <=0;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.13s/rows]


,f0_
0,5828994


In [11]:
print(f"Total percentage of null values : {100* (5828994 / 53108721)}")

Total percentage of null values : 10.975587229826152


In [13]:
%%bigquery

SELECT 
    count(*) as num_trips,
    MIN(tripduration) / 60 as min_trip_duration_minutes, 
    max(tripduration) / 60 as max_trip_duration_minutes
FROM
    `bigquery-public-data.new_york_citibike.citibike_trips`
WHERE
    tripduration is not NULL AND tripduration > 0

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.05rows/s]


,num_trips,min_trip_duration_minutes,max_trip_duration_minutes
0,53108721,1.0,325167.483333


In [14]:
%%bigquery
SELECT 
    EXTRACT (YEAR FROM starttime) as year,
    EXTRACT (MONTH FROM starttime) as month,
    count(*) total
FROM
    `bigquery-public-data.new_york_citibike.citibike_trips`
WHERE
    (tripduration >= 300 AND tripduration <= 10800) AND
    EXTRACT (YEAR FROM starttime) = 2017 OR 
    EXTRACT (YEAR FROM starttime) = 2018 AND 
    birth_year is not NULL AND 
    birth_year < 2007
GROUP BY
    year, month
ORDER BY
    year, month ASC

Downloading: 100%|██████████| 14/14 [00:01<00:00, 12.86rows/s]


,year,month,total
0,2017,4,1103779
1,2017,5,1282373
2,2017,6,1464462
3,2017,7,1471769
4,2017,8,1540029
5,2017,9,1592627
6,2017,10,1583536
7,2017,11,1073609
8,2017,12,699228
9,2018,1,718994


## Split the dataset into Training, Evaluation and Prediction tables

In [15]:
%%bigquery
CREATE OR REPLACE TABLE `nyc_bike_share.training_table` AS
SELECT
    tripduration/60 tripduration,
    starttime,
    stoptime,
    start_station_id,
    start_station_name,
    start_station_latitude,
    start_station_longitude,
    end_station_id,
    end_station_name,
    end_station_latitude,
    end_station_longitude,
    bikeid,
    usertype,
    birth_year,
    gender,
    customer_plan
FROM
    `bigquery-public-data.new_york_citibike.citibike_trips`
WHERE
    (
        (EXTRACT (YEAR FROM starttime)=2017 AND
        (EXTRACT (MONTH FROM starttime)>=4 OR EXTRACT (MONTH FROM starttime)<=12))
    OR 
        (EXTRACT (YEAR FROM starttime)=2018 AND
        (EXTRACT (MONTH FROM starttime)>=1 OR EXTRACT (MONTH FROM starttime)<=2))
    )
    AND (tripduration>=300 AND tripduration<=10800)
    AND birth_year is not NULL
    AND birth_year < 2007;

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 713.11query/s]                         


""


In [16]:
%%bigquery
SELECT COUNT(*) FROM `nyc_bike_share.training_table`

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.01rows/s]


,f0_
0,14936821


* tripduration greater than `5 minutes` and less than `3 hours`
* month number 3 and 4 for year 2018
* birth year less than 2017 and not null

In [17]:
%%bigquery

CREATE OR REPLACE TABLE `nyc_bike_share.evaluation_table` AS
SELECT
    tripduration/60 tripduration,
    starttime,
    stoptime,
    start_station_id,
    start_station_name,
    start_station_latitude,
    start_station_longitude,
    end_station_id,
    end_station_name,
    end_station_latitude,
    end_station_longitude,
    bikeid,
    usertype,
    birth_year,
    gender,
    customer_plan
FROM
    `bigquery-public-data.new_york_citibike.citibike_trips`
WHERE
    (EXTRACT (YEAR FROM starttime)=2018 AND
    (EXTRACT (MONTH FROM starttime)=3 OR EXTRACT (MONTH FROM starttime)=4))
    AND (tripduration>=300 AND tripduration<=10800)
    AND birth_year is not NULL
    AND birth_year < 2007;

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1555.56query/s]                        


""


In [19]:
%%bigquery
SELECT COUNT(*) FROM `nyc_bike_share.evaluation_table`

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.00rows/s]


,f0_
0,1819395


* tripduration greater than `5 minutes` and less than `3 hours`

In [21]:
%%bigquery

CREATE OR REPLACE TABLE `nyc_bike_share.prediction_table` AS
SELECT
    tripduration/60 tripduration,
    starttime,
    stoptime,
    start_station_id,
    start_station_name,
    start_station_latitude,
    start_station_longitude,
    end_station_id,
    end_station_name,
    end_station_latitude,
    end_station_longitude,
    bikeid,
    usertype,
    birth_year,
    gender,
    customer_plan
FROM
    `bigquery-public-data.new_york_citibike.citibike_trips`
WHERE
    EXTRACT (YEAR FROM starttime) = 2018
    AND EXTRACT (MONTH FROM starttime) = 5
    AND (tripduration >= 300 AND tripduration <= 10800) 
    AND birth_year is not NULL
    AND birth_year < 2007;

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1182.05query/s]                        


""


In [22]:
%%bigquery
SELECT COUNT(*) FROM `nyc_bike_share.prediction_table`

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.03rows/s]


,f0_
0,1527475


In [49]:
Training =  14936821 
Evaluation = 1819395
Prediction = 1527475

In [50]:
total = (Training + Evaluation + Prediction)
total

18283691

In [42]:
print(f"Training data : {100 * Training / total}% ")

Training data : 81.69477924342519% 


In [41]:
print(f"Evaluation data : {100 * Evaluation / total}% ") 

Evaluation data : 9.950917459718609% 


In [40]:
print(f"Prediction data : {100 * Prediction / total}% ")

Prediction data : 8.354303296856198% 


In [36]:
%%bigquery

CREATE OR REPLACE MODEL `nyc_bike_share.trip_duration_xgb_predictor`
TRANSFORM (* EXCEPT(starttime, stoptime, start_station_id, end_station_id, bikeid),
    ML.BUCKETIZE(EXTRACT(HOUR FROM starttime), [5, 10, 17]) AS hourofday,
    EXTRACT(WEEK FROM starttime) AS starttime_week,
    EXTRACT(MONTH FROM starttime) AS starttime_month,
    IF (EXTRACT(DAYOFWEEK FROM starttime) BETWEEN 2 and 6, true, false) AS is_weekend,
    (EXTRACT(YEAR FROM starttime) - birth_year) AS age,
    ST_DISTANCE(
        ST_GEOGPOINT(start_station_longitude, start_station_latitude), 
        ST_GEOGPOINT(end_station_longitude, end_station_latitude)) AS distance)
OPTIONS
    (MODEL_TYPE='BOOSTED_TREE_REGRESSOR',
    INPUT_LABEL_COLS = ['tripduration'],
    BOOSTER_TYPE = 'GBTREE',
    NUM_PARALLEL_TREE = 1,
    MAX_ITERATIONS = 50,
    TREE_METHOD = 'HIST',
    EARLY_STOP = TRUE,
    ENABLE_GLOBAL_EXPLAIN = TRUE) AS
SELECT
    *
FROM
    `nyc_bike_share.training_table`

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1155.24query/s]                        


""


In [46]:
%%bigquery
SELECT
    *
FROM
    ML.TRAINING_INFO(MODEL `nyc_bike_share.trip_duration_xgb_predictor`)

Downloading: 100%|██████████| 9/9 [00:01<00:00,  7.99rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,9,7.58681,6.92653,0.3,48
1,0,8,7.62988,6.96996,0.3,62
2,0,7,7.70329,7.04674,0.3,44
3,0,6,7.84217,7.18918,0.3,43
4,0,5,8.09599,7.44654,0.3,60
5,0,4,8.57691,7.94520,0.3,13964
6,0,3,9.46915,8.86985,0.3,53
7,0,2,11.05230,10.51290,0.3,66
8,0,1,13.72140,13.27020,0.3,212410


In [47]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL `nyc_bike_share.trip_duration_xgb_predictor`)

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.03s/rows]


,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,3.656444,47.976859,0.097913,2.172709,0.535788,0.538238


In [39]:
%%bigquery

SELECT 
    *
FROM 
    ML.EVALUATE(MODEL `nyc_bike_share.trip_duration_xgb_predictor`, (
                 SELECT * FROM `nyc_bike_share.evaluation_table`
                ))

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.02rows/s]


,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,4.005738,70.328708,0.106598,2.220785,0.485437,0.487123


In [38]:
%%bigquery

SELECT 
    predicted_tripduration, 
    tripduration, 
    start_station_name, 
    end_station_name
FROM 
    ML.PREDICT(MODEL `nyc_bike_share.trip_duration_xgb_predictor`, (
                 SELECT * FROM `nyc_bike_share.prediction_table`
                ))

Downloading: 100%|██████████| 1527475/1527475 [00:01<00:00, 770988.57rows/s] 


,predicted_tripduration,tripduration,start_station_name,end_station_name
0,8.419283,5.716667,E 85 St & York Ave,E 85 St & 3 Ave
1,7.541323,8.150000,Fulton St & Waverly Ave,Clinton Ave & Myrtle Ave
2,8.483127,14.600000,5 Ave & E 29 St,Lexington Ave & E 24 St
3,11.305532,8.616667,Pershing Square South,Broadway & E 22 St
4,7.808872,6.516667,Broadway & W 29 St,E 24 St & Park Ave S
...,...,...,...,...
1527470,7.206043,6.133333,Broadway & W 37 St,Broadway & W 25 St
1527471,7.206043,5.200000,St James Pl & Oliver St,Fulton St & William St
1527472,7.206043,6.533333,31 Ave & 14 St,31 St & Broadway
1527473,7.206043,5.233333,31 Ave & 14 St,31 St & Broadway
